In [11]:
import os

def enviorment_check(id):
    dir = '/Users/christsai/Google 雲端硬碟/US_Stock_Prediction/history/'
    if os.path.exists(dir) == False:
        raise IndexError('Can not find the default folder')
    save_folder = str(dir) + str(id) 
    os.makedirs(save_folder)
    return (save_folder + '/')

In [12]:
# define get data function

import pandas as pd
import numpy as np
import quandl

def GetStockData_PriceVolume(paras):
    '''
    All data is from quandl wiki dataset
    Feature set: [Open  High    Low  Close    Volume  Ex-Dividend  Split Ratio Adj. Open  Adj. High  Adj. Low  
    Adj. Close  Adj. Volume]
    '''
    
    # Prepare data frame
    stkname = "WIKI/" + str(paras.ticker)
    df = quandl.get(stkname, authtoken = '2c24stWyXfdzLVFWxGe4', start_date = paras.start_date, end_date = paras.end_date)
    df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]
    df = df.rename(columns = {"Adj. Open": "open", "Adj. High": "high", "Adj. Low": "low",
                            "Adj. Close": "close", "Adj. Volume": "volume"})
    df_all = df.copy()
    df['MA'] = df['close'].rolling(window = paras.pred_len, center = False).mean()
    #df['MA'] = pd.DataFrame({'MA': df['close'].rolling(window = paras['pred_len'],center = False).mean() })
    
    # Data frame output
    if paras.out_class_type == 'regression':
        if paras.out_type == 'MA':
            df['label'] = df['MA'].shift(-1 * paras.pred_len)
        else:
            df['label'] = df['close'].shift(-1 * paras.pred_len)
    else: # classification - FIXME
        df['label'] = df['close'].shift(-1 * paras.pred_len)
    
    # Generate input features for time series data
    featureset = list(['label'])
    featuresDict = {'c':'close', 'h':'high', 'l':'low', 'o':'open', 'v':'volume'}
    for i in range(paras.window_len, -1, -1):
        for j in list(paras.features):
            df[j+'_-'+str(i)+'_d'] = df[featuresDict[j]].shift(1*i)
            featureset.append(j+'_-'+str(i)+'_d')
    
    df = df[featureset]
    df_lately = df[-paras.pred_len:]
    df.dropna(inplace = True)
    df_known_lately = df[len(df) - paras.known_lately_len: len(df)]
    df = df[0:len(df)-paras.known_lately_len]

    if paras.debug_load_data == True:
        pd.set_option('display.max_columns', None)
        print (' %%%%%%%%%%%%%%% GetStockData PriceVolume Parameters Set %%%%%%%%%%%%%%% ')
        print (paras)
        print (' %%%%%%%%%%%%%%% GetStockData PriceVolume Feature Set %%%%%%%%%%%%%%% ')
        print (featureset)
        print (' %%%%%%%%%%%%%%% GetStockData PriceVolume df Statistic %%%%%%%%%%%%%%% ')
        print (df.describe())
        print (' %%%%%%%%%%%%%%% GetStockData PriceVolume df_known_lately Statistic %%%%%%%%%%%%%%% ')
        print (df_known_lately.describe())
        print (' %%%%%%%%%%%%%%% GetStockData PriceVolume df_lately Statistic %%%%%%%%%%%%%%% ')
        print (df_lately.describe())
    
    return df, df_known_lately, df_lately, df_all
    

In [13]:
# normalization/ preprocessing
from sklearn import preprocessing

def preprocessing_data_by_row(data, paras):
    '''
    data: N*M np.array
    N: sample
    M: features
    data_T: M*N
    data_T_scale: scaler for column by column, M*N
    data_T_scale_T: N*M
    '''
    if data.size == 0:
        return data, None
    
    data_T = data.transpose()
    if paras.preproc_scaler == 'standard_scaler':
        scaler = preprocessing.StandardScaler().fit(data_T)
    else: #FIXME
        scaler = preprocessing.StandardScaler().fit(data_T)
    data_T_scale = scaler.transform(data_T)
    data_T_scale_T = data_T_scale.transpose()
    return data_T_scale_T, scaler

In [14]:
def divide_into_price_volume(df, paras):
    '''
    df.columns = [..., o_-10_d,h_-10_d,l_-10_d,c_-10_d,v_-10_d,...]
    return [...,o_-10_d,h_-10_d,l_-10_d,c_-10_d,...], [...,v_-10_d,...]
    '''
    volume_cols = [col for col in df.columns if 'v_' in col]
    return np.array(df.drop(volume_cols, 1)), np.array(df[volume_cols])

In [15]:
def preprocessing_data(df, paras, featureDropForTraining, withLabel = True):
    '''
    df: pd.DataFrame
    X: np.array
    y: np.array
    convert df into X,y
    '''
    y = np.array(df['label'])
    X_price, X_volume = divide_into_price_volume(df.drop(featureDropForTraining, 1), paras)
    
    X_price, scaler_price = preprocessing_data_by_row(X_price, paras)
    X_volume, scaler_volume = preprocessing_data_by_row(X_volume, paras)
    
    # combine price and volume - rearrange
    # [...,o_-10_d,h_-10_d,l_-10_d,c_-10_d,...], [...,v_-10_d,...] -> [..., o_-10_d,h_-10_d,l_-10_d,c_-10_d,v_-10_d,...]
    X_combined = X_price
    if len(X_volume[0]) != 0:
        for i in range(len(X_volume[0])-1, -1, -1):
            X_combined = np.insert(X_combined, (i+1)*(paras.n_features-1), X_volume[:,i], axis = 1)

    if withLabel == True:
        y_normalized = scaler_price.transform(y.reshape(1,-1))    
        y_normalized_T = y_normalized.reshape(-1,1)
    else:
        y_normalized_T = np.repeat(float('nan'), len(y))
    scaler_combined = {'price': scaler_price, 'volume': scaler_volume}
    
    if paras.debug_preproc_data == True:
        print ('df[-1:]:',df[-1:])
        
        if scaler_price != None:
            print ('scaler_price len:', np.shape(scaler_price.mean_))
            print ('scaler_price mean:',scaler_price.mean_)
            print ('scaler_price var:',scaler_price.var_)
            print ('normalized price:',X_price[-1:])
            print ('inversed price:',scaler_price.inverse_transform(X_price.transpose())[:,-1])

        if scaler_volume != None:
            print ('scaler_volume len:', np.shape(scaler_volume.mean_))
            print ('scaler_volume mean:',scaler_volume.mean_)
            print ('scaler_volume var:',scaler_volume.var_)
            print ('normalized volume:',X_volume[-1:])
            print ('inversed volume:',scaler_volume.inverse_transform(X_volume.transpose())[:,-1])

        print ('X_price shape',X_price.shape)
        print ('X_volume shape',X_volume.shape)
        print ('X_combined shape',X_combined.shape)
        print ('X_combined',X_combined[-1:])
        
        print ('y[-1:]:', y[-1:])
        print ('y[-1] normalized:', y_normalized[-1:])
        print ('y[-1] inversed:', scaler_price.inverse_transform(y_normalized)[:,-1])
    
    return X_combined, y_normalized_T, scaler_combined

In [16]:
def reshape_input(X, y, paras):
    n_sample = X.shape[0]
    n_channel = paras.n_features
    n_features_per_channel = X.shape[1]/n_channel
    X_reshaped = np.reshape(X, (n_sample, n_features_per_channel, n_channel))
    y_reshaped = np.reshape(y, (n_sample,-1))
    return X_reshaped, y_reshaped

In [17]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

def build_LSTM_model(paras):
    model_lstm = Sequential()
    first = True
    for idx in range(len(paras.model['hidden_layers'])):
        if idx == (len(paras.model['hidden_layers']) - 1):
            model_lstm.add(LSTM(paras.model['hidden_layers'][idx], return_sequences = False))
            model_lstm.add(Activation(paras.model['activation'][idx]))
            model_lstm.add(Dropout(paras.model['dropout'][idx]))
        elif first == True:
            model_lstm.add(LSTM(input_dim = paras.n_features,
                                output_dim = paras.model['hidden_layers'][idx],
                                return_sequences = True))
            model_lstm.add(Activation(paras.model['activation'][idx]))
            model_lstm.add(Dropout(paras.model['dropout'][idx]))
            first = False
        else:
            model_lstm.add(LSTM(paras.model['hidden_layers'][idx], return_sequences = True))
            model_lstm.add(Activation(paras.model['activation'][idx]))
            model_lstm.add(Dropout(paras.model['dropout'][idx]))
   
    # output layer
    model_lstm.add(Dense(output_dim = paras.model['out_layer']))
    model_lstm.add(Activation(paras.model['out_activation']))
    model_lstm.compile(loss = paras.model['loss'], optimizer = paras.model['optimizer'])
    print ('build LSTM model...')
    return model_lstm

Using TensorFlow backend.


In [18]:
def LSTM_model_predict(model, X, y, scaler = None):
    predictions = model.predict(X)
    mse_scaled = np.mean((y - predictions) ** 2)
    print ('scaled data mse: ', mse_scaled)
    
    if scaler != None:
        arr = np.array(scaler.inverse_transform(y.reshape(y.shape[0],)))
        arr2 = np.array(scaler.inverse_transform(predictions.reshape(predictions.shape[0],)))
        return mse_scaled, arr, arr2
    return mse_scaled, None, None

In [19]:
def save_training_model(model, paras, name):
    # https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model
    model.save(paras.save_folder + name + '.h5')  # creates a HDF5 file 'my_model.h5'

In [20]:
from datetime import timedelta
def append_date_serires(df, paras):
    append_date = []
    append_last_date = df.index[-1]
    i = paras.pred_len
    while i >= 1:
        append_last_date = append_last_date + timedelta(days=1)
        if append_last_date.isoweekday() > 0 and append_last_date.isoweekday() < 6:
            append_date.append(append_last_date)
            i -= 1
    append_df = pd.DataFrame(index = list(append_date))
    df = pd.concat((df, append_df), axis=0)
    return df

In [21]:
def save_data_frame_mse(df, paras, mses):
    df = append_date_serires(df, paras)
    
    if 1:
        df['actual'] = df['actual']
        df['pred'] = df['pred']
        df = df.rename(columns = {"actual": "a_+"+str(paras.pred_len)+'_d',
                                  "pred": "p_+"+str(paras.pred_len)+'_d'})
        
        df['a_+'+str(paras.pred_len)+'_d_diff'] = df["a_+"+str(paras.pred_len)+'_d'] - df['close']
        df['p_+'+str(paras.pred_len)+'_d_diff'] = df["p_+"+str(paras.pred_len)+'_d'] - df['close']
        new_list = ["a_+"+str(paras.pred_len)+'_d', "p_+"+str(paras.pred_len)+'_d', 
                    'a_+'+str(paras.pred_len)+'_d_diff', 'p_+'+str(paras.pred_len)+'_d_diff']
    else :
        # shift +
        df['actual'] = df['actual'].shift(paras.pred_len)
        df['pred'] = df['pred'].shift(paras.pred_len)
        df = df.rename(columns = {"actual": "a_-"+str(paras.pred_len)+'_d',
                                  "pred": "p_-"+str(paras.pred_len)+'_d'})
        new_list = ["a_-"+str(paras.pred_len)+'_d', "p_-"+str(paras.pred_len)+'_d']
    
    default_list = ['open', 'high', 'low', 'close', 'volume']
    original_other_list = set(df.columns) - set(default_list) - set(new_list)
    original_other_list = list(original_other_list)
    df = df[default_list + original_other_list + new_list]
    model_acc = mses[1] / mses[0]
    df.to_csv(paras.save_folder + paras.ticker + ('_%.2f' % model_acc)+ "_data_frame.csv")
    with open(paras.save_folder + 'parameters.txt', "w") as text_file:
        text_file.write(paras.__str__())
        text_file.write(str(mses[0]) + '\n')
        text_file.write(str(mses[1]) + '\n')
    return df

In [22]:
# plot
# http://matplotlib.org/examples/pylab_examples/subplots_demo.html

import matplotlib.pyplot as plt
import csv
    
def plot_training_curve(history, paras, save = True):
    # LSTM model plot
    # from keras.utils.visualize_util import plot
    # from IPython.display import SVG
    # from keras.utils.visualize_util import model_to_dot
    # plot(model_lstm, to_file='model.png')
    # SVG(model_to_dot(model_lstm).create(prog='dot', format='svg'))
    
    %matplotlib inline
    # Control the default size of figures in this Jupyter notebook
    %pylab inline
    pylab.rcParams['figure.figsize'] = (15, 9)   # Change the size of plots

    # LSTM training 
    f, ax = plt.subplots()
    ax.plot(history.history['loss'])
    ax.plot(history.history['val_loss'])
    ax.set_title('loss function')
    ax.set_ylabel('mse')
    ax.set_xlabel('epoch')
    ax.legend(['loss', 'val_loss'], loc='upper right')
    plt.show()
    if save == True:
        savefig(paras.save_folder + 'training_curve.png')
        w = csv.writer(open(paras.save_folder + "training_curve_model.txt", "w"))
        for key, val in history.history.items():
            w.writerow([key, val])
        for key, val in history.params.items():
            w.writerow([key, val])   

In [23]:
# thanks for : https://ntguardian.wordpress.com/2016/09/19/introduction-stock-market-data-python-1/
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY
from matplotlib.finance import candlestick_ohlc

def plot_result(df_total, paras, save = True):
#     df_total = pd.concat((df[-paras.window_len:], df_known_lately, df_lately), axis = 0)
#     df_total = append_date_serires(df_total, paras)
    
#     df_total['pred'] = df_total['pred'].shift(paras.pred_len)
#     df_total['val'] = df_total['val'].shift(paras.pred_len)
    
    mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
    alldays = DayLocator()              # minor ticks on the days
    dayFormatter = DateFormatter('%d')      # e.g., 12
    
    # plot candleStick/ line for all data
    # Set plot parameters, including the axis object ax used for plotting
    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=0.2)
    if df_total.index[-1] - df_total.index[0] < pd.Timedelta('730 days'):
        weekFormatter = DateFormatter('%b %d')  # e.g., Jan 12
        ax.xaxis.set_major_locator(mondays)
        ax.xaxis.set_minor_locator(alldays)
    else:
        weekFormatter = DateFormatter('%b %d, %Y')
    ax.xaxis.set_major_formatter(weekFormatter)
    ax.grid(True)
    
    if set(['h','l','c','o']) <= set(list(paras.features)):
        # plot candle stick
        # Create the candelstick chart
        candlestick_ohlc(ax, list(zip(list(date2num(df_total.index.tolist())), 
                                      df_total["o_-0_d"].tolist(), df_total["h_-0_d"].tolist(),
                                      df_total["l_-0_d"].tolist(), df_total["c_-0_d"].tolist())),
                         colorup = "green", colordown = "red", width = 0.4)
    else:
        # plot line
        df_total.loc[:,['c_-0_d']].plot(ax = ax, lw = 1.3, grid = True, legend = ['close'])

    # plot prediction and true value
    df_total.loc[:,['pred', 'val']].plot(ax = ax, lw = 1.3, grid = True, legend = ['pred', 'val'])

    ax.xaxis_date()
    ax.autoscale_view()
    plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')
    plt.ion()
    
    
    # add annotations
    ax.annotate('known lately start', (date2num(df_known_lately.index.tolist()[0]), 
                                       df_known_lately["c_-0_d"].tolist()[0]), 
                xytext=(15, 15), textcoords='offset points', 
                arrowprops=dict(arrowstyle='-|>'))
    
    ax.annotate('lately start', (date2num(df_lately.index.tolist()[0]), 
                                       df_lately["c_-0_d"].tolist()[0]), 
                xytext=(15, 15), textcoords='offset points', 
                arrowprops=dict(arrowstyle='-|>'))
        
    plt.show()
    
    if save == True:
        savefig(paras.save_folder + 'pred_result.png')


In [24]:
from sklearn import cross_validation
from keras.callbacks import History 

def run_LSTM(paras):
    ################################################################################
    paras.save_folder = enviorment_check(paras.identify)
    print ('save directory: ', paras.save_folder)
    ################################################################################

    featureDropForTraining = ['label']

    # get data
    df, df_known_lately, df_lately, df_all = GetStockData_PriceVolume(paras)

    # preprocessing
    X, y, scaler = preprocessing_data(df, paras, featureDropForTraining, withLabel = True)
    X_known_lately, y_known_lately, scaler_known_lately = preprocessing_data(df_known_lately, paras, featureDropForTraining, withLabel = True)
    X_lately, y_lately, scaler_lately = preprocessing_data(df_lately, paras, featureDropForTraining, withLabel = False)

    # cross validation
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)
    print ('train shape',X_train.shape)
    print ('test shape',X_test.shape)

    # reshape input data to LSTM model
    X_train, y_train = reshape_input(X_train, y_train, paras)
    X_test, y_test = reshape_input(X_test, y_test, paras)
    X_known_lately, y_known_lately = reshape_input(X_known_lately, y_known_lately, paras)
    X_lately, y_lately = reshape_input(X_lately, y_lately, paras)

    # build LSTM model
    history = History()
    model_lstm = build_LSTM_model(paras)
    model_lstm.fit(
        X_train,
        y_train,
        batch_size = paras.batch_size,
        nb_epoch = paras.epoch,
        validation_split = paras.validation_split,
        #validation_data = (X_known_lately, y_known_lately),
        callbacks=[history],
        verbose = 1
    )
    # save model
    save_training_model(model_lstm, paras, 'lstm_model')

    # validation test + known lately data 
    print (' ############## validation on test data ############## ')
    mse_test, tmp, tmp2 = LSTM_model_predict(model_lstm, X_test, y_test)

    print (' ############## validation on known lately data ############## ')
    mse_known_lately, df_all.loc[df_known_lately.index, 'actual'], df_all.loc[df_known_lately.index, 'pred'] = LSTM_model_predict(model_lstm, X_known_lately, y_known_lately, 
                                                      scaler = scaler_known_lately['price'])
    # predict lately data
    print (' ############## validation on lately data ############## ')
    mse_lately, df_all.loc[df_lately.index, 'actual'], df_all.loc[df_lately.index, 'pred'] = LSTM_model_predict(model_lstm, X_lately, y_lately, 
                                       scaler = scaler_lately['price'])

    # rewrite data frame and save / update
    df_all = save_data_frame_mse(df_all, paras, mses = [mse_test, mse_known_lately])

    # plot training loss/ validation loss
    plot_training_curve(history, paras, save = True)

    # # plot result
    # show_len = paras.window_len + paras.known_lately_len + paras.pred_len*2
    # plot_result(df_all[-show_len:], paras, save = True)

In [ ]:
import US_SP_Global_Parameters
if __name__ == "__main__":
    # #################### parameters setting ####################
    paras = US_SP_Global_Parameters.SP_RNN_LSTM_Paras('LSTM', ticker = 'NDAQ')
    paras.features = 'ohlcv'
    paras.window_len = 120
    paras.pred_len = 20
    paras.known_lately_len = 20
    paras.debug_load_data = False
    paras.debug_preproc_data = False

    # LSTM parameters
    paras.batch_size = 128
    paras.epoch = 50
    paras.validation_split = 0.1
    paras.model['hidden_layers'] = [120, 60, 30]
    paras.model['dropout'] = [0.5, 0.5, 0.3]
    paras.model['activation'] = ['relu', 'relu', 'relu']
    paras.model['out_layer'] = 1
    paras.model['out_activation'] = 'linear'
    paras.model['loss'] = 'mse'
    paras.model['optimizer'] = 'rmsprop'

    run_LSTM(paras)





save directory:  /Users/christsai/Google 雲端硬碟/US_Stock_Prediction/history/LSTM_NDAQ_2017-03-03 15:30:10/
train shape (1315, 605)
test shape (329, 605)


/Users/christsai/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:225: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)


build LSTM model...
Train on 1183 samples, validate on 132 samples
Epoch 1/50
1183/1183 [==============================] - 23s - loss: 1.6694 - val_loss: 1.0310
Epoch 2/50
1183/1183 [==============================] - 19s - loss: 1.0379 - val_loss: 0.8203
Epoch 3/50
1183/1183 [==============================] - 19s - loss: 0.8494 - val_loss: 1.3285
Epoch 4/50
1183/1183 [==============================] - 20s - loss: 0.8681 - val_loss: 0.7523
Epoch 5/50
 512/1183 [===========>..................] - ETA: 11s - loss: 0.6707